# Lesson 5: Stock Market Analysis with LLM Feedback Loops - SOLUTION

## Understanding LLM-to-LLM Feedback Systems

This notebook demonstrates the core concept of LLM feedback loops where:
1. **Analyst LLM** generates initial stock analysis
2. **Critic LLM** evaluates and provides specific feedback
3. **Analyst LLM** refines its analysis based on critic feedback
4. Process repeats until quality criteria are met

**Time Required**: 15-20 minutes
**Context**: Stock market analysis and investment recommendations

## Setup and Configuration

In [1]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import time

# Load environment variables
load_dotenv('../../.env')

# Setup OpenAI client for Vocareum environment
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ Environment setup complete")

✅ Environment setup complete


## Demo Scenario: Apple Stock Analysis

We'll analyze Apple (AAPL) stock using a two-LLM feedback system:
- **Stock Analyst LLM**: Generates investment analysis and recommendations
- **Investment Critic LLM**: Reviews and provides improvement feedback

In [2]:
# Sample market data for Apple (AAPL) - simplified for demo
market_data = """
Company: Apple Inc. (AAPL)
Current Price: $175.50
52-Week Range: $124.17 - $199.62
Market Cap: $2.75T
P/E Ratio: 28.5
Recent News: 
- iPhone 15 launch exceeded expectations
- Services revenue grew 16% YoY
- China sales down 4% due to local competition
- New Vision Pro headset launching next year
- Strong cash position of $162B
"""

print("📊 Market data loaded for Apple (AAPL)")
print(market_data)

📊 Market data loaded for Apple (AAPL)

Company: Apple Inc. (AAPL)
Current Price: $175.50
52-Week Range: $124.17 - $199.62
Market Cap: $2.75T
P/E Ratio: 28.5
Recent News: 
- iPhone 15 launch exceeded expectations
- Services revenue grew 16% YoY
- China sales down 4% due to local competition
- New Vision Pro headset launching next year
- Strong cash position of $162B



## Step 1: Define the LLM Agents

In [3]:
# Stock Analyst LLM - Generates initial analysis
ANALYST_PROMPT = """
You are a Senior Stock Analyst at a major investment firm. Your job is to analyze stocks and provide investment recommendations.

Analyze the provided stock data and create a brief investment recommendation (2-3 paragraphs) that includes:
1. Key strengths and risks
2. Price target and recommendation (Buy/Hold/Sell)
3. Investment rationale

Be concise but thorough in your analysis.
"""

# Investment Critic LLM - Evaluates and provides feedback
CRITIC_PROMPT = """
You are a Senior Investment Critic and Risk Manager. Your job is to review stock analyses and provide constructive feedback.

Review the stock analysis and provide specific feedback on:
1. Missing critical factors or risks
2. Unclear or unsupported claims
3. Areas that need more depth or evidence
4. Overall quality score (1-10)

If the analysis scores 8+ and addresses key factors comprehensively, say "APPROVED".
Otherwise, provide specific improvement suggestions.
"""

print("🤖 LLM Agents defined:")
print("- Stock Analyst: Generates investment analysis")
print("- Investment Critic: Reviews and provides feedback")

🤖 LLM Agents defined:
- Stock Analyst: Generates investment analysis
- Investment Critic: Reviews and provides feedback


## Step 2: Helper Functions for LLM Interaction

In [4]:
def get_analyst_analysis(market_data, previous_feedback=None):
    """Get stock analysis from the Analyst LLM"""
    
    user_prompt = f"Market Data:\n{market_data}"
    
    if previous_feedback:
        user_prompt += f"\n\nPrevious Feedback to Address:\n{previous_feedback}"
        user_prompt += "\n\nPlease revise your analysis to address the feedback above."
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": ANALYST_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    
    return response.choices[0].message.content

def get_critic_feedback(analysis, market_data):
    """Get feedback from the Critic LLM"""
    
    user_prompt = f"""
Original Market Data:
{market_data}

Stock Analysis to Review:
{analysis}

Please provide your critical review and feedback.
"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": CRITIC_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
        max_tokens=400
    )
    
    return response.choices[0].message.content

print("🔧 Helper functions created for LLM interactions")

🔧 Helper functions created for LLM interactions


## Step 3: Demonstration - Single Feedback Loop

In [5]:
print("🚀 Starting LLM Feedback Loop Demo...\n")
print("="*60)
print("ROUND 1: Initial Analysis")
print("="*60)

# Step 1: Get initial analysis from Analyst LLM
initial_analysis = get_analyst_analysis(market_data)

print("📈 ANALYST OUTPUT:")
print("-" * 40)
print(initial_analysis)
print()

🚀 Starting LLM Feedback Loop Demo...

ROUND 1: Initial Analysis
📈 ANALYST OUTPUT:
----------------------------------------
**Investment Recommendation for Apple Inc. (AAPL)**

Apple Inc. exhibits several key strengths that make it a compelling investment opportunity. The recent launch of the iPhone 15 has exceeded market expectations, indicating strong consumer demand and effective brand loyalty. Additionally, the growth in services revenue by 16% year-over-year underlines Apple's successful diversification beyond hardware sales, contributing to a more stable revenue stream. The company's robust cash position of $162 billion provides a cushion for future investments and shareholder returns, which could include buybacks or dividends. However, risks remain, particularly with a 4% decline in sales from China due to increasing local competition, which poses a threat to one of Apple’s vital markets.

Given these factors, I recommend a **Buy** rating for AAPL with a price target of **$200**.

In [6]:
# Step 2: Get feedback from Critic LLM
print("🔍 CRITIC REVIEW:")
print("-" * 40)

critic_feedback = get_critic_feedback(initial_analysis, market_data)
print(critic_feedback)
print()

# Check if approved
is_approved = "APPROVED" in critic_feedback.upper()
print(f"✅ Status: {'APPROVED' if is_approved else 'NEEDS REVISION'}")

🔍 CRITIC REVIEW:
----------------------------------------
**Critical Review of Stock Analysis for Apple Inc. (AAPL)**

1. **Missing Critical Factors or Risks:**
   - The analysis does not address potential macroeconomic risks such as inflation, interest rate changes, or supply chain disruptions that could impact Apple's operations and profitability.
   - There is no mention of regulatory risks, particularly in China and the EU, where antitrust actions could affect Apple's business practices and revenue.
   - The analysis overlooks the potential impact of currency fluctuations, especially given Apple's significant international sales.

2. **Unclear or Unsupported Claims:**
   - The claim that the iPhone 15 launch exceeded expectations is vague. It would benefit from specific sales figures or comparisons to previous launches to substantiate this assertion.
   - The statement about the P/E ratio being "slightly above the industry average" lacks context. It would be helpful to provide the 

## Step 4: Feedback Loop - Iterative Improvement

In [7]:
if not is_approved:
    print("\n" + "="*60)
    print("ROUND 2: Revised Analysis Based on Feedback")
    print("="*60)
    
    # Step 3: Get revised analysis incorporating feedback
    revised_analysis = get_analyst_analysis(market_data, critic_feedback)
    
    print("📈 REVISED ANALYST OUTPUT:")
    print("-" * 40)
    print(revised_analysis)
    print()
    
    # Step 4: Get final review
    print("🔍 FINAL CRITIC REVIEW:")
    print("-" * 40)
    
    final_feedback = get_critic_feedback(revised_analysis, market_data)
    print(final_feedback)
    print()
    
    final_approved = "APPROVED" in final_feedback.upper()
    print(f"✅ Final Status: {'APPROVED' if final_approved else 'NEEDS FURTHER REVISION'}")
else:
    print("\n🎉 Analysis approved on first iteration!")


ROUND 2: Revised Analysis Based on Feedback
📈 REVISED ANALYST OUTPUT:
----------------------------------------
**Investment Recommendation for Apple Inc. (AAPL)**

Apple Inc. (AAPL) presents an attractive investment opportunity, driven by its strong product portfolio, particularly bolstered by the successful iPhone 15 launch and robust growth in its services segment, which increased 16% year-over-year. The company boasts a solid cash position of $162 billion, providing ample liquidity for innovation and potential acquisitions. However, investors should be mindful of the 4% decline in sales from China due to intensifying local competition, which could pose a long-term threat to market share in one of its largest markets. Additionally, macroeconomic risks such as inflation, rising interest rates, and ongoing supply chain disruptions, as well as regulatory pressures in China and the EU, may impact profitability.

Considering these factors, we recommend a **Hold** rating for AAPL with a p

## Step 5: Complete Feedback Loop Function

In [8]:
def stock_analysis_with_feedback(market_data, max_iterations=3):
    """
    Complete LLM feedback loop for stock analysis
    
    Returns:
        dict: Final analysis results and iteration history
    """
    
    results = {
        'iterations': [],
        'final_analysis': None,
        'approved': False,
        'total_iterations': 0
    }
    
    analysis = None
    feedback = None
    
    for iteration in range(max_iterations):
        print(f"\n🔄 Iteration {iteration + 1}")
        print("-" * 30)
        
        # Get analysis (initial or revised)
        analysis = get_analyst_analysis(market_data, feedback)
        
        # Get critic feedback
        feedback = get_critic_feedback(analysis, market_data)
        
        # Store iteration results
        results['iterations'].append({
            'analysis': analysis,
            'feedback': feedback,
            'approved': "APPROVED" in feedback.upper()
        })
        
        # Check if approved
        if "APPROVED" in feedback.upper():
            print("✅ Analysis APPROVED!")
            results['approved'] = True
            break
        else:
            print("🔄 Needs revision, continuing...")
    
    results['final_analysis'] = analysis
    results['total_iterations'] = len(results['iterations'])
    
    return results

print("🔧 Complete feedback loop function ready")

🔧 Complete feedback loop function ready


## Step 6: Test Different Scenarios

In [9]:
# Test with a more challenging scenario
challenging_data = """
Company: Tesla Inc. (TSLA)
Current Price: $248.50
52-Week Range: $138.80 - $299.29
Market Cap: $790B
P/E Ratio: 65.2
Recent News:
- Q3 deliveries missed expectations by 2%
- Cybertruck production delays continue
- FSD (Full Self Driving) beta expanding
- Price cuts in China market
- Musk selling more shares for Twitter
- Competition increasing from Ford, GM
"""

print("🧪 Testing with more challenging scenario: Tesla (TSLA)")
print("This scenario has mixed signals and should trigger more feedback loops...\n")

tesla_results = stock_analysis_with_feedback(challenging_data, max_iterations=2)

print(f"\n📊 Summary:")
print(f"- Total iterations: {tesla_results['total_iterations']}")
print(f"- Final approval: {tesla_results['approved']}")
print(f"- Improvement demonstrated: {'Yes' if tesla_results['total_iterations'] > 1 else 'No'}")

🧪 Testing with more challenging scenario: Tesla (TSLA)
This scenario has mixed signals and should trigger more feedback loops...


🔄 Iteration 1
------------------------------
🔄 Needs revision, continuing...

🔄 Iteration 2
------------------------------
🔄 Needs revision, continuing...

📊 Summary:
- Total iterations: 2
- Final approval: False
- Improvement demonstrated: Yes


## Key Concepts Demonstrated

### 1. LLM-to-LLM Feedback Loop
- **Analyst LLM** generates initial analysis
- **Critic LLM** evaluates and provides specific feedback
- **Analyst LLM** incorporates feedback for improvement
- Process continues until quality criteria are met

### 2. Real-World Applications
- **Investment Research**: Multi-step analysis refinement
- **Risk Assessment**: Peer review and validation
- **Report Generation**: Quality control and improvement
- **Decision Support**: Collaborative AI reasoning

### 3. Benefits of This Approach
- **Quality Improvement**: Iterative refinement leads to better outputs
- **Error Reduction**: Critic LLM catches missing factors or errors
- **Consistency**: Systematic evaluation criteria
- **Transparency**: Clear improvement trail and reasoning

## Summary

This solution illustrated the core concept of **LLM feedback loops** in financial analysis:

✅ **LLM-to-LLM Communication**: One LLM evaluates another's output  
✅ **Iterative Improvement**: Analysis gets refined through feedback cycles  
✅ **Quality Control**: Systematic evaluation criteria and approval process  
✅ **Real-World Application**: Stock analysis with practical financial context  

**Key Takeaway**: Unlike traditional ML training, this approach uses LLMs to provide contextual, reasoning-based feedback that leads to immediate output improvement - demonstrating true AI-to-AI collaboration in financial decision-making.

---
*Time: ~15-20 minutes | Context: Stock Market Analysis | Concept: LLM Feedback Loops*